Project 007 - Jupyter Notebook Code  
Andy Snitgen  
Professor Karen Jin  
Comp 574 - Applied Computing II  
October 31, 2021  

In [1]:
import requests, json, time, csv, pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

# Create the data frame
column_names = ['Title', 'Year', 'Actor', 'US Box Office', 'International Revenue', 'Adjusted Revenue', 'Average Rating', 'IMDb Rating', 'Metacritic']
df = pd.DataFrame(columns = column_names)

In [2]:
# Create a function that grabs the relevant data and returns the new row
def data(movie_title):
    # Set initial API string for IMDB
    api_key = "k_nh5thpq8"
    base_url = "https://imdb-api.com/en/API/SearchMovie/"
    complete_url = base_url + api_key + "/" + movie_title

    # Get the title ID
    payload = requests.get(complete_url).json()
    df_temp = pd.json_normalize(payload['results'])
    id_num = df_temp.id.iloc[0]

    # use the id number to get the required info
    base_url = "https://imdb-api.com/en/API/Title/"
    complete_url = base_url + api_key + "/" + id_num 
    payload = requests.get(complete_url).json()
    df_temp = pd.json_normalize(payload)
    df_temp2 = pd.json_normalize(payload['starList'])

    # Extract relevant data and put it into variables
    title = df_temp.title.iloc[0]
    year = df_temp.year.iloc[0]
    actor = df_temp2.name.iloc[0]
    us_box = df_temp['boxOffice.grossUSA'].iloc[0]
    itl_box = df_temp["boxOffice.cumulativeWorldwideGross"].iloc[0]
    critic = payload['metacriticRating']
    critic = critic + '%'
    rating = payload['imDbRating']
    if rating == '':
        rating = critic
        rating = rating.replace('%', '')
        rating = float(rating)
        rating = rating / 10
        rating = str(rating) + ' / 10 (est.)'
    else:
        rating = rating + ' / 10'

    # Check if International Box Office Data exists, if it doesn't replace the value with 'No Data'
    if title == 'No Time to Die' and checker() == False:  # No Time to Die has to be put in manually as API doesn't have data yet
        us_box = '$120,357,453'
        itl_box = '$525,482,453'
    us_box_float = floater(us_box)
    itl_box_float =  floater(itl_box)
    if abs((us_box_float - itl_box_float) / us_box_float) <= 0.01:
        itl_box = "No Data"

    # Put variables into main data frame
    new_row = {'Title': title, 'Year': year, 'Actor': actor, 'US Box Office': us_box, "International Revenue": itl_box, 'IMDb Rating': rating, 'Metacritic': critic}
    return new_row
     

In [3]:
# Create a function that converts $ strings into floats
def floater(input_str):
    floater = input_str.strip('$')
    floater = floater.replace(',','')
    floater = floater.replace(' (est.)', '')
    floater = float(floater)
    return floater

In [4]:
# Create a function that turns an IMDb rating string into a float
def replace_rating(input_str):
    ret_value = input_str.replace(' / 10', '')
    ret_value = ret_value.replace(' (est.)', '')
    ret_value = float(ret_value)
    ret_value = ret_value * 10
    return ret_value

In [5]:
# Create a function that turns a Metacritic score into a float
def replace_critic(input_str):
    ret_value = input_str.replace('%', '')
    ret_value = float(ret_value)
    return ret_value

In [6]:
# function to test if 'No Time to Die' has data yet
def checker():
    movie_title = 'no time to die'

    # Set initial API string for IMDB
    api_key = "k_nh5thpq8"
    base_url = "https://imdb-api.com/en/API/SearchMovie/"
    complete_url = base_url + api_key + "/" + movie_title

    # Get the title ID
    payload = requests.get(complete_url).json()
    df_temp = pd.json_normalize(payload['results'])
    id_num = df_temp.id.iloc[0]

    # use the id number to get the required info
    base_url = "https://imdb-api.com/en/API/Title/"
    complete_url = base_url + api_key + "/" + id_num 
    payload = requests.get(complete_url).json()
    df_temp = pd.json_normalize(payload)

    # Extract relevant data and put it into variables
    us_box = df_temp['boxOffice.grossUSA'].iloc[0]
    
    # Return the boolean result of the checker()
    if us_box != '':
        return True
    else: 
        return False

In [7]:
# Create the list of movies to collect data on
movie_list = \
['dr no',
'from russia with love',
'goldfinger',
'thunderball', 
'you only live twice',
'on her majestys secret service',
'diamonds are forever',
'live and let die',
'the man with the golden gun',
'the spy who loved me',
'moonraker',
'for your eyes only',
'octopussy',
'view to a kill',
'the living daylights',
'licence to kill',
'goldeneye',
'tomorrow never dies',
'the world is not enough',
'die another day',
'casino royale',
'quantum of solace',
'skyfall',
'spectre',
'no time to die']

In [8]:
# Run checker() to determine if No Time to Die is up to date in the API
print("No Time to Die has up to date Box Office API data: ")
print(checker())
if checker() == True:
    print("The movie's ticket sales will come from IMDb's API")
elif checker() == False:
    print("The movie's Box Office data will be hard-coded in")

No Time to Die has up to date Box Office API data: 
False
The movie's Box Office data will be hard-coded in


In [9]:
# These commented out lines are use for debugging
# movie_list = movie_list[20:25] 
# print(movie_list)


# Iterate through the list and add results to data frame 
for movie_title in movie_list:
    print('working on ' + movie_title)
    time.sleep(1)  # Put into reduce hang up error on home PC
    new_row = data(movie_title)
    df = df.append(new_row, ignore_index=True)

working on dr no
working on from russia with love
working on goldfinger
working on thunderball
working on you only live twice
working on on her majestys secret service
working on diamonds are forever
working on live and let die
working on the man with the golden gun
working on the spy who loved me
working on moonraker
working on for your eyes only
working on octopussy
working on view to a kill
working on the living daylights
working on licence to kill
working on goldeneye
working on tomorrow never dies
working on the world is not enough
working on die another day
working on casino royale
working on quantum of solace
working on skyfall
working on spectre
working on no time to die


In [10]:
# Calculate the Average change from Domestic to International Gross 
total = 0
for index, row in df.iterrows():
    if df['International Revenue'].values[index] != 'No Data':
        usbo = floater(str(df['US Box Office'].values[index]))
        itlbo = floater(str(df['International Revenue'].values[index]))
        total = total + (itlbo/usbo)
counter = df['International Revenue'].str.contains("No Data").value_counts()[0]
change = total/counter

# Apply the average change by multiplying US Box Office by 'change' variable to replace 'No Data'
for index, row in df.iterrows():
    if df['International Revenue'].values[index] == 'No Data':
        usbo = floater(str(df['US Box Office'].values[index]))
        itlbo = usbo*change
        itlbo = "${:,.0f} (est.)".format(itlbo)
        df['International Revenue'][index] = itlbo

In [11]:
# Scrape inflation data from the internet
page = requests.get('https://www.minneapolisfed.org/about-us/monetary-policy/inflation-calculator/consumer-price-index-1913-')
soup = BeautifulSoup(page.content, 'html.parser')
data = soup.find('tbody')
tables = data.find_all('td')
info_list = []
for elem in tables:
    info = elem.findChildren()[0].contents[0]
    info_list.append(info)

# Put the data into lists and combine the lists into a dictionary
year = []
cpi = []
for value in info_list[::3]:
    value = value.replace('*', '')
    value = value.replace(u'\xa0', '')
    year.append(value)
for value in info_list[1::3]:
    cpi.append(value)
zip_iterator = zip(year, cpi)
inf_dict = dict(zip_iterator)

In [12]:
# Calculate inflation for each movie and place it into a new column 'Adjusted Revenue'
curr_cpi = floater(inf_dict.get('2021'))
for index, row in df.iterrows():
    key = df['Year'].values[index]
    old_cpi = floater(inf_dict.get(key))
    base = floater(df['International Revenue'].values[index])
    adjusted = base * (curr_cpi/old_cpi)
    adjusted = "${:,.0f}".format(adjusted)
    df['Adjusted Revenue'][index] = adjusted

In [13]:
# Calculate the average of IMDb and Metacritic ratings and put it into new column Average Rating
for index, row in df.iterrows():
    rate = replace_rating(df['IMDb Rating'].values[index])
    crit = replace_critic(df['Metacritic'].values[index])
    average = (rate + crit)/2
    average = str(average) + '%'
    df['Average Rating'][index] = average
df

,Title,Year,Actor,US Box Office,International Revenue,Adjusted Revenue,Average Rating,IMDb Rating,Metacritic
0,Dr. No,1962,Sean Connery,"$16,067,035","$56,367,173 (est.)","$504,886,163",75.0%,7.2 / 10,78%
1,From Russia with Love,1963,Sean Connery,"$24,796,765","$86,993,247 (est.)","$771,567,557",79.0%,7.4 / 10,84%
2,Goldfinger,1964,Sean Connery,"$51,081,062","$179,205,128 (est.)","$1,568,911,992",82.0%,7.7 / 10,87%
3,Thunderball,1965,Sean Connery,"$63,595,658","$223,109,458 (est.)","$1,922,282,759",67.0%,7 / 10,64%
4,You Only Live Twice,1967,Sean Connery,"$43,084,787","$151,152,198 (est.)","$1,228,224,747",65.0%,6.9 / 10,61%
5,On Her Majesty's Secret Service,1969,George Lazenby,"$22,774,493","$79,898,612 (est.)","$590,857,855",64.0%,6.7 / 10,61%
6,Diamonds Are Forever,1971,Sean Connery,"$43,819,547","$153,729,919 (est.)","$1,030,180,247",62.5%,6.6 / 10,59%
7,Live and Let Die,1973,Roger Moore,"$35,377,836","$124,114,288 (est.)","$758,662,562",61.5%,6.8 / 10,55%
8,The Man with the Golden Gun,1974,Roger Moore,"$20,972,000","$73,575,016 (est.)","$405,035,686",55.0%,6.7 / 10,43%
9,The Spy Who Loved Me,1977,Roger Moore,"$46,838,673","$164,321,768 (est.)","$735,922,902",63.0%,7.1 / 10,55%


In [14]:
# Write the data frame to a csv file
f = open('project007.csv', 'w', newline='')
writer = csv.writer(f)
writer.writerow(['Project 007 CSV', 'By: Andy Snitgen', 'For: Karen Jin', 'Class: comp574', 'Date: Oct. 31, 2021'])
writer.writerow(['Title', 'Year', 'Actor', 'US Box Office', 'International Revenue', 'Adjusted Revenue', 'Average Rating', 'IMDb Rating', 'Metacritic']) 
for index, row in df.iterrows():
    title = df.Title[index]
    year = df.Year[index]
    actor = df.Actor[index]
    us_box_office = df['US Box Office'][index]
    international_revenue = df['International Revenue'][index]
    adjusted_revenue = df['Adjusted Revenue'][index]
    average_rating = df['Average Rating'][index]
    imdb_rating = df['IMDb Rating'][index]
    metacritic = df['Metacritic'][index]
    writer.writerow([title, year, actor, us_box_office, international_revenue, adjusted_revenue, average_rating, imdb_rating, metacritic])
f.close()